In [1]:
import psycopg2 as postgres_connector
import mysql.connector as mysql_connector
from mysql.connector import errorcode
import sqlite3
import os, sys
sys.path.append("..")
from utility.sql_utilities import SQLUtilities as ut

In [2]:
DATABASE_NAME: str = "db_onlinestore"

## PostgreSQL

In [3]:
dbconfig = {"user":os.environ['POSTGRESQL_USERNAME'], "password": os.environ['POSTGRESQL_PASSWORD'], "host":"localhost", "port":5432}
try:
    postgres_connection = postgres_connector.connect(**dbconfig)
    print("Connection established between Postgres Database and Python")
    postgres_connection.autocommit = True
    postgres_cursor = postgres_connection.cursor()
except:
    print("Something is wrong with your user name or password")
    

# Connect to a database. In Postgres, you must connect to a database
exists: bool = ut.database_exists(DATABASE_NAME, cursor_object=postgres_cursor)
if exists:
    ut.execute_query(query=f"""DROP DATABASE {DATABASE_NAME}""", cursor_object=postgres_cursor)
ut.execute_query(query=f'CREATE DATABASE {DATABASE_NAME}', cursor_object=postgres_cursor)
dbconfig["database"] = DATABASE_NAME

try:
    postgres_connection = postgres_connector.connect(**dbconfig)
    print("Connection established between PostgresSQL Database and Python")
    postgres_connection.autocommit = True
    postgres_cursor = postgres_connection.cursor()
except:
    print("Something is wrong with your user name or password")

ut.display_all_tables_in_database(cursor_object=postgres_cursor)

Connection established between Postgres Database and Python
Query ran successfully in time: (0.125 sec)
Query ran successfully in time: (0.227 sec)
Connection established between PostgresSQL Database and Python
+------------+--------------+
| table_name | table_schema |
+------------+--------------+
+------------+--------------+
0 rows returned in time: (0.006 sec)




In [4]:
ut.execute_display_query_results(query="SELECT version();", cursor_object=postgres_cursor)

+------------------------------------------------------------+
|                          version                           |
+------------------------------------------------------------+
| PostgreSQL 15.0, compiled by Visual C++ build 1914, 64-bit |
+------------------------------------------------------------+
1 row returned in time: (0.0 sec)




## MySQL

In [5]:
try:
    mysql_connection = mysql_connector.connect(host="localhost", user=os.environ['MYSQL_USERNAME'], password=os.environ['MYSQL_PASSWORD'], port=3306, autocommit=True)
    print("Connection established between MySQL Database and Python")
    mysql_cursor = mysql_connection.cursor()
    print("Cursor object created to interact with MySQL Server")
except mysql_connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  else:
    print(err) 

exists: bool = ut.database_exists(DATABASE_NAME, cursor_object=mysql_cursor)
if exists:
      ut.execute_query(query=f"""DROP DATABASE {DATABASE_NAME}""", cursor_object=mysql_cursor)
ut.execute_query(query=f'CREATE DATABASE {DATABASE_NAME}', cursor_object=mysql_cursor)

#Check that database was created 
assert ut.database_exists(database_name=DATABASE_NAME, cursor_object=mysql_cursor) == True

# Set the newly created database as the database to use
ut.execute_query(query=f"USE {DATABASE_NAME}", cursor_object=mysql_cursor)
ut.display_all_tables_in_database(cursor_object=mysql_cursor)

Connection established between MySQL Database and Python
Cursor object created to interact with MySQL Server
Query ran successfully in time: (0.178 sec)
Query ran successfully in time: (0.012 sec)
Query ran successfully in time: (0.001 sec)
+------------+---------------+---------------+
| TABLE_NAME | DATABASE NAME | TABLE_CATALOG |
+------------+---------------+---------------+
+------------+---------------+---------------+
0 rows returned in time: (0.002 sec)




## SQLite

In [6]:
sqlite_connection = sqlite3.connect(f'{DATABASE_NAME}.db')
sqlite_cursor = sqlite_connection.cursor()

In [7]:
## Serial means auto generated
## CASCADE will drop any foreign key that is referencing the table in postgres
ut.execute_query(query="""SET FOREIGN_KEY_CHECKS=0;""", cursor_object=mysql_cursor)

delete_query: str = """ DROP TABLE IF EXISTS tbl_categories {};"""
ut.execute_query(query=delete_query.format("CASCADE"), cursor_object=postgres_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=mysql_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=sqlite_cursor)

create_table_query: str = """
CREATE TABLE IF NOT EXISTS tbl_categories (category_id INTEGER PRIMARY KEY {}, name VARCHAR(100) NOT NULL)
"""
ut.execute_query(query=create_table_query.format("GENERATED ALWAYS AS IDENTITY"), cursor_object=postgres_cursor)
ut.execute_query(query=create_table_query.format("AUTO_INCREMENT"), cursor_object=mysql_cursor)
ut.execute_query(query=create_table_query.format(""), cursor_object=sqlite_cursor)



delete_query: str = """ DROP TABLE IF EXISTS tbl_products {};"""
ut.execute_query(query=delete_query.format("CASCADE"), cursor_object=postgres_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=mysql_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=sqlite_cursor)

create_table_query: str = """
CREATE TABLE IF NOT EXISTS tbl_products (product_id INTEGER PRIMARY KEY {}, name VARCHAR(100) NOT NULL, 
price DECIMAL(10, 2), description VARCHAR(255), tags VARCHAR(255), category_id INT,supplier VARCHAR(100),
FOREIGN KEY (category_id) REFERENCES tbl_categories (category_id));
"""
ut.execute_query(query=create_table_query.format("GENERATED ALWAYS AS IDENTITY"), cursor_object=postgres_cursor)
ut.execute_query(query=create_table_query.format("AUTO_INCREMENT"), cursor_object=mysql_cursor)
ut.execute_query(query=create_table_query.format(""), cursor_object=sqlite_cursor)



delete_query: str = """ DROP TABLE IF EXISTS tbl_customers {};"""
ut.execute_query(query=delete_query.format("CASCADE"), cursor_object=postgres_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=mysql_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=sqlite_cursor)

create_table_query: str = """
CREATE TABLE IF NOT EXISTS tbl_customers (customer_id INTEGER PRIMARY KEY {}, customer_name VARCHAR(100) NOT NULL, 
email VARCHAR(100) NOT NULL, phone_number VARCHAR(20), address VARCHAR(255), city VARCHAR(255));
"""
ut.execute_query(query=create_table_query.format("GENERATED ALWAYS AS IDENTITY"), cursor_object=postgres_cursor)
ut.execute_query(query=create_table_query.format("AUTO_INCREMENT"), cursor_object=mysql_cursor)
ut.execute_query(query=create_table_query.format(""), cursor_object=sqlite_cursor)


delete_query: str = """ DROP TABLE IF EXISTS tbl_orders {};"""
ut.execute_query(query=delete_query.format("CASCADE"), cursor_object=postgres_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=mysql_cursor)
ut.execute_query(query=delete_query.format(""), cursor_object=sqlite_cursor)

create_table_query: str = """
CREATE TABLE IF NOT EXISTS tbl_orders (order_id INTEGER PRIMARY KEY {}, customer_id INT, product_id INT, total_quantity INT, 
total_amount DECIMAL(10, 2), order_rating DECIMAL(3,1), length DECIMAL(5, 2), width DECIMAL(5, 2), order_timestamp TIMESTAMP, delivery_timestamp TIMESTAMP,
FOREIGN KEY (customer_id) REFERENCES tbl_customers (customer_id),
FOREIGN KEY (product_id) REFERENCES tbl_products (product_id));
"""
ut.execute_query(query=create_table_query.format("GENERATED ALWAYS AS IDENTITY"), cursor_object=postgres_cursor)
ut.execute_query(query=create_table_query.format("AUTO_INCREMENT"), cursor_object=mysql_cursor)
ut.execute_query(query=create_table_query.format(""), cursor_object=sqlite_cursor)

# Enable the foreign key checks
ut.execute_query(query="""SET FOREIGN_KEY_CHECKS=1;""", cursor_object=mysql_cursor)

print("------------Postgres------------")
ut.display_all_tables_in_database(cursor_object=postgres_cursor)

print("\n------------MySQL------------")
ut.display_all_tables_in_database(cursor_object=mysql_cursor)

print("\n------------SQLite------------")
ut.display_all_tables_in_database(cursor_object=sqlite_cursor)

Query ran successfully in time: (0.0 sec)
Query ran successfully in time: (0.001 sec)
Query ran successfully in time: (0.004 sec)
Query ran successfully in time: (0.017 sec)
Query ran successfully in time: (0.014 sec)
Query ran successfully in time: (0.049 sec)
Query ran successfully in time: (0.015 sec)
Query ran successfully in time: (0.0 sec)
Query ran successfully in time: (0.006 sec)
Query ran successfully in time: (0.014 sec)
Query ran successfully in time: (0.014 sec)
Query ran successfully in time: (0.054 sec)
Query ran successfully in time: (0.014 sec)
Query ran successfully in time: (0.0 sec)
Query ran successfully in time: (0.008 sec)
Query ran successfully in time: (0.014 sec)
Query ran successfully in time: (0.013 sec)
Query ran successfully in time: (0.039 sec)
Query ran successfully in time: (0.015 sec)
Query ran successfully in time: (0.001 sec)
Query ran successfully in time: (0.003 sec)
Query ran successfully in time: (0.015 sec)
Query ran successfully in time: (0.011

In [8]:
insert_query = """
INSERT INTO tbl_categories (name) VALUES ('Electronics'), ('Clothing'), ('Home and Kitchen')
"""
ut.execute_query(query=insert_query, cursor_object=postgres_cursor)
ut.execute_query(query=insert_query, cursor_object=mysql_cursor)
ut.execute_query(query=insert_query, cursor_object=sqlite_cursor)

print("------------Postgres------------")
ut.select_all_query(table_name='tbl_categories', cursor_object=postgres_cursor)

print("\n------------MySQL------------")
ut.select_all_query(table_name='tbl_categories', cursor_object=mysql_cursor)

print("\n------------MySQLite------------")
ut.select_all_query(table_name='tbl_categories', cursor_object=sqlite_cursor)

Query ran successfully in time: (0.003 sec)
Query ran successfully in time: (0.013 sec)
Query ran successfully in time: (0.001 sec)
------------Postgres------------
+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitchen |
+-------------+------------------+
3 rows returned in time: (0.0 sec)



------------MySQL------------
+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitchen |
+-------------+------------------+
3 rows returned in time: (0.001 sec)



------------MySQLite------------
+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitc

In [9]:
insert_query = """
INSERT INTO tbl_products (name, price, description, tags, category_id, supplier) 
VALUES 
('Laptop', NULL, 'High-performance laptop for professionals', 'electronics, portable, tech', 1, 'SupplierA'), 
('Headphones', 129.99, 'Over-ear wireless headphones', 'electronics, audio, accessories', 1, 'SupplierB'), 
('Backpack', 49.99, 'Stylish and durable backpack for everyday use', 'fashion, accessories, travel', 2, 'NULL'), 
('Coffee Maker', NULL, 'Automatic drip coffee maker with programmable', 'home, kitchen, applicances', 3, 'NULL'), 
('Fitness Tracker', 79.99, 'Water-resistant fitness tracker with heart rate monitor', 'electronics, fitness, wearables', 1, 'NULL'); 
"""
ut.execute_query(query=insert_query, cursor_object=postgres_cursor)
ut.execute_query(query=insert_query, cursor_object=mysql_cursor)
ut.execute_query(query=insert_query, cursor_object=sqlite_cursor)

print("------------Postgres------------")
ut.select_all_query(table_name='tbl_products', cursor_object=postgres_cursor)


print("\n------------MySQL------------")
ut.select_all_query(table_name='tbl_products', cursor_object=mysql_cursor)

print("\n------------SQLite------------")
ut.select_all_query(table_name='tbl_products', cursor_object=sqlite_cursor)

Query ran successfully in time: (0.005 sec)
Query ran successfully in time: (0.013 sec)
Query ran successfully in time: (0.001 sec)
------------Postgres------------
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
| product_id |      name       | price  |                       description                       |              tags               | category_id | supplier  |
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
|     1      |     Laptop      |  NULL  |        High-performance laptop for professionals        |   electronics, portable, tech   |      1      | SupplierA |
|     2      |   Headphones    | 129.99 |              Over-ear wireless headphones               | electronics, audio, accessories |      1      | SupplierB |
|     3      |    Backpack     | 49

In [10]:
insert_query = """
INSERT INTO tbl_customers (customer_name, email, phone_number, address, city) 
VALUES 
('Alice Johnson', 'alice@example.com', '123-456-7890', '123 Main St', 'Cityville'), 
('Bob Smith', 'bob@example.com', '987-654-3210', '456 Oak Ave', 'Townsville'), 
('Charlie Brown', 'charlie@example.com', '555-123-4567', '789 Pine ln', 'New York'), 
('David White', 'david@example.com', '246-357-9753', '567 Maple Dr', 'NYC'), 
('Eva Black', 'eva@example.com', '789-864-5791', '890 Cedar Rd', 'Big Apple'),
('Frank Green', 'frank@example.com', '579-468-375', '123 Elm St', 'Gotham City'); 
"""
ut.execute_query(query=insert_query, cursor_object=postgres_cursor)
ut.execute_query(query=insert_query, cursor_object=mysql_cursor)
ut.execute_query(query=insert_query, cursor_object=sqlite_cursor)

print("------------Postgres------------")
ut.select_all_query(table_name='tbl_customers', cursor_object=postgres_cursor)


print("\n------------MySQL------------")
ut.select_all_query(table_name='tbl_customers', cursor_object=mysql_cursor)

print("\n------------SQLite------------")
ut.select_all_query(table_name='tbl_customers', cursor_object=sqlite_cursor)

Query ran successfully in time: (0.002 sec)
Query ran successfully in time: (0.01 sec)
Query ran successfully in time: (0.0 sec)
------------Postgres------------
+-------------+---------------+---------------------+--------------+--------------+-------------+
| customer_id | customer_name |        email        | phone_number |   address    |    city     |
+-------------+---------------+---------------------+--------------+--------------+-------------+
|      1      | Alice Johnson |  alice@example.com  | 123-456-7890 | 123 Main St  |  Cityville  |
|      2      |   Bob Smith   |   bob@example.com   | 987-654-3210 | 456 Oak Ave  | Townsville  |
|      3      | Charlie Brown | charlie@example.com | 555-123-4567 | 789 Pine ln  |  New York   |
|      4      |  David White  |  david@example.com  | 246-357-9753 | 567 Maple Dr |     NYC     |
|      5      |   Eva Black   |   eva@example.com   | 789-864-5791 | 890 Cedar Rd |  Big Apple  |
|      6      |  Frank Green  |  frank@example.com  | 

In [11]:
insert_query = """
INSERT INTO tbl_orders (customer_id, product_id, total_quantity, total_amount, order_rating, length, width, order_timestamp, delivery_timestamp) 
VALUES 
(1, 1, 5, 120.50, 4.5, 2.3, 1.8, '2023-01-15 10:30:00', '2023-01-16 15:45:00'), 
(2, 2, 3, 75.25, 3.8, 1.5, 1.2, '2023-02-03 14:20:00', '2023-02-05 11:10:00'), 
(3, 3, 7, 210.75, 4.2, 2.8, 2.0, '2023-03-12 08:45:00', '2023-03-14 09:30:00'), 
(1, 4, 2, 50.00, 4.0, 1.8, 1.5, '2023-04-05 12:15:00', '2023-04-07 18:20:00'); 
"""
ut.execute_query(query=insert_query, cursor_object=postgres_cursor)
ut.execute_query(query=insert_query, cursor_object=mysql_cursor)
ut.execute_query(query=insert_query, cursor_object=sqlite_cursor)

print("------------Postgres------------")
ut.select_all_query(table_name='tbl_orders', cursor_object=postgres_cursor)


print("\n------------MySQL------------")
ut.select_all_query(table_name='tbl_orders', cursor_object=mysql_cursor)

print("\n------------MySQLite------------")
ut.select_all_query(table_name='tbl_orders', cursor_object=sqlite_cursor)

Query ran successfully in time: (0.004 sec)
Query ran successfully in time: (0.05 sec)
Query ran successfully in time: (0.0 sec)
------------Postgres------------
+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
| order_id | customer_id | product_id | total_quantity | total_amount | order_rating | length | width |   order_timestamp   | delivery_timestamp  |
+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
|    1     |      1      |     1      |       5        |    120.50    |     4.5      |  2.30  | 1.80  | 2023-01-15 10:30:00 | 2023-01-16 15:45:00 |
|    2     |      2      |     2      |       3        |    75.25     |     3.8      |  1.50  | 1.20  | 2023-02-03 14:20:00 | 2023-02-05 11:10:00 |
|    3     |      3      |     3      |       7        |    210.75    |     4.2      |  2.80  | 2.

In [12]:
query_string: str = "SELECT * FROM tbl_customers WHERE customer_name LIKE 'A%'";
ut.execute_display_query_results(query=query_string, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_string, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=query_string, cursor_object=sqlite_cursor)

+-------------+---------------+-------------------+--------------+-------------+-----------+
| customer_id | customer_name |       email       | phone_number |   address   |   city    |
+-------------+---------------+-------------------+--------------+-------------+-----------+
|      1      | Alice Johnson | alice@example.com | 123-456-7890 | 123 Main St | Cityville |
+-------------+---------------+-------------------+--------------+-------------+-----------+
1 row returned in time: (0.0 sec)



+-------------+---------------+-------------------+--------------+-------------+-----------+
| customer_id | customer_name |       email       | phone_number |   address   |   city    |
+-------------+---------------+-------------------+--------------+-------------+-----------+
|      1      | Alice Johnson | alice@example.com | 123-456-7890 | 123 Main St | Cityville |
+-------------+---------------+-------------------+--------------+-------------+-----------+
1 row returned in time: (0.001 se

In [13]:
query_string: str = "SELECT * FROM tbl_customers ORDER BY customer_name DESC;"
ut.execute_display_query_results(query=query_string, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_string, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=query_string, cursor_object=sqlite_cursor)

+-------------+---------------+---------------------+--------------+--------------+-------------+
| customer_id | customer_name |        email        | phone_number |   address    |    city     |
+-------------+---------------+---------------------+--------------+--------------+-------------+
|      6      |  Frank Green  |  frank@example.com  | 579-468-375  |  123 Elm St  | Gotham City |
|      5      |   Eva Black   |   eva@example.com   | 789-864-5791 | 890 Cedar Rd |  Big Apple  |
|      4      |  David White  |  david@example.com  | 246-357-9753 | 567 Maple Dr |     NYC     |
|      3      | Charlie Brown | charlie@example.com | 555-123-4567 | 789 Pine ln  |  New York   |
|      2      |   Bob Smith   |   bob@example.com   | 987-654-3210 | 456 Oak Ave  | Townsville  |
|      1      | Alice Johnson |  alice@example.com  | 123-456-7890 | 123 Main St  |  Cityville  |
+-------------+---------------+---------------------+--------------+--------------+-------------+
6 rows returned in t

In [14]:
select_query = "SELECT * FROM tbl_products WHERE price > 100 AND category_id = 1;"
ut.execute_display_query_results(query=select_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=select_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=select_query, cursor_object=sqlite_cursor)

+------------+------------+--------+------------------------------+---------------------------------+-------------+-----------+
| product_id |    name    | price  |         description          |              tags               | category_id | supplier  |
+------------+------------+--------+------------------------------+---------------------------------+-------------+-----------+
|     2      | Headphones | 129.99 | Over-ear wireless headphones | electronics, audio, accessories |      1      | SupplierB |
+------------+------------+--------+------------------------------+---------------------------------+-------------+-----------+
1 row returned in time: (0.001 sec)



+------------+------------+--------+------------------------------+---------------------------------+-------------+-----------+
| product_id |    name    | price  |         description          |              tags               | category_id | supplier  |
+------------+------------+--------+-----------------------------

In [15]:
ut.show_columns(table_name="tbl_customers", cursor_object=postgres_cursor)
print("\n-----------------------MySQL-----------------------")
ut.show_columns(table_name="tbl_customers", cursor_object=mysql_cursor)
print("\n-----------------------SQLite-----------------------")
ut.show_columns(table_name="tbl_customers", cursor_object=sqlite_cursor)

+---------------+-------------------+-------------+-------------+----------------+-------------+
|  column_name  |     data_type     | is_nullable | is_identity | column_default | primary_key |
+---------------+-------------------+-------------+-------------+----------------+-------------+
|  customer_id  |      integer      |     NO      |     YES     |      NULL      |     YES     |
| customer_name | character varying |     NO      |     NO      |      NULL      |     NO      |
|     email     | character varying |     NO      |     NO      |      NULL      |     NO      |
| phone_number  | character varying |     YES     |     NO      |      NULL      |     NO      |
|    address    | character varying |     YES     |     NO      |      NULL      |     NO      |
|     city      | character varying |     YES     |     NO      |      NULL      |     NO      |
+---------------+-------------------+-------------+-------------+----------------+-------------+
6 rows returned in time: (0.01

In [16]:
select_query = "SELECT * FROM tbl_customers WHERE city = 'New York' ORDER BY customer_name;"
ut.execute_display_query_results(query=select_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=select_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=select_query, cursor_object=sqlite_cursor)

+-------------+---------------+---------------------+--------------+-------------+----------+
| customer_id | customer_name |        email        | phone_number |   address   |   city   |
+-------------+---------------+---------------------+--------------+-------------+----------+
|      3      | Charlie Brown | charlie@example.com | 555-123-4567 | 789 Pine ln | New York |
+-------------+---------------+---------------------+--------------+-------------+----------+
1 row returned in time: (0.0 sec)



+-------------+---------------+---------------------+--------------+-------------+----------+
| customer_id | customer_name |        email        | phone_number |   address   |   city   |
+-------------+---------------+---------------------+--------------+-------------+----------+
|      3      | Charlie Brown | charlie@example.com | 555-123-4567 | 789 Pine ln | New York |
+-------------+---------------+---------------------+--------------+-------------+----------+
1 row returned in time:

In [17]:
ut.select_all_query(table_name="tbl_products", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=mysql_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=sqlite_cursor)

+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
| product_id |      name       | price  |                       description                       |              tags               | category_id | supplier  |
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
|     1      |     Laptop      |  NULL  |        High-performance laptop for professionals        |   electronics, portable, tech   |      1      | SupplierA |
|     2      |   Headphones    | 129.99 |              Over-ear wireless headphones               | electronics, audio, accessories |      1      | SupplierB |
|     3      |    Backpack     | 49.99  |      Stylish and durable backpack for everyday use      |  fashion, accessories, travel   |      2      |   NULL    |
|     4      |  Coffee Maker   |  NULL  

In [18]:
update_query = "UPDATE tbl_products SET price = 500 WHERE product_id = 4;"
postgres_cursor.execute(update_query)
mysql_cursor.execute(update_query)
ut.select_all_query(table_name="tbl_products", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=mysql_cursor)

+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
| product_id |      name       | price  |                       description                       |              tags               | category_id | supplier  |
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
|     1      |     Laptop      |  NULL  |        High-performance laptop for professionals        |   electronics, portable, tech   |      1      | SupplierA |
|     2      |   Headphones    | 129.99 |              Over-ear wireless headphones               | electronics, audio, accessories |      1      | SupplierB |
|     3      |    Backpack     | 49.99  |      Stylish and durable backpack for everyday use      |  fashion, accessories, travel   |      2      |   NULL    |
|     5      | Fitness Tracker | 79.99  

In [19]:
update_query = "UPDATE tbl_products SET price = 550 WHERE product_id = 4;"
postgres_cursor.execute(update_query)
mysql_cursor.execute(update_query)
sqlite_cursor.execute(update_query)
ut.select_all_query(table_name="tbl_products", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=mysql_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=sqlite_cursor)

+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
| product_id |      name       | price  |                       description                       |              tags               | category_id | supplier  |
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
|     1      |     Laptop      |  NULL  |        High-performance laptop for professionals        |   electronics, portable, tech   |      1      | SupplierA |
|     2      |   Headphones    | 129.99 |              Over-ear wireless headphones               | electronics, audio, accessories |      1      | SupplierB |
|     3      |    Backpack     | 49.99  |      Stylish and durable backpack for everyday use      |  fashion, accessories, travel   |      2      |   NULL    |
|     5      | Fitness Tracker | 79.99  

In [20]:
ut.select_all_query(table_name="tbl_orders", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_orders", cursor_object=mysql_cursor)

+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
| order_id | customer_id | product_id | total_quantity | total_amount | order_rating | length | width |   order_timestamp   | delivery_timestamp  |
+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
|    1     |      1      |     1      |       5        |    120.50    |     4.5      |  2.30  | 1.80  | 2023-01-15 10:30:00 | 2023-01-16 15:45:00 |
|    2     |      2      |     2      |       3        |    75.25     |     3.8      |  1.50  | 1.20  | 2023-02-03 14:20:00 | 2023-02-05 11:10:00 |
|    3     |      3      |     3      |       7        |    210.75    |     4.2      |  2.80  | 2.00  | 2023-03-12 08:45:00 | 2023-03-14 09:30:00 |
|    4     |      1      |     4      |       2        |    50.00     |     4.0      |  1.80  | 1.50  | 2023-04-

## Joins

In [21]:
join_query = """
SELECT name, description, total_amount from tbl_orders o
INNER JOIN tbl_products p USING(product_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)

+--------------+-----------------------------------------------+--------------+
|     name     |                  description                  | total_amount |
+--------------+-----------------------------------------------+--------------+
|    Laptop    |   High-performance laptop for professionals   |    120.50    |
|  Headphones  |         Over-ear wireless headphones          |    75.25     |
|   Backpack   | Stylish and durable backpack for everyday use |    210.75    |
| Coffee Maker | Automatic drip coffee maker with programmable |    50.00     |
+--------------+-----------------------------------------------+--------------+
4 rows returned in time: (0.0 sec)



+--------------+-----------------------------------------------+--------------+
|     name     |                  description                  | total_amount |
+--------------+-----------------------------------------------+--------------+
|    Laptop    |   High-performance laptop for professionals   |    120.50    |
| 

In [22]:
join_query = """
SELECT customer_name, name, description, total_amount from tbl_orders o
INNER JOIN tbl_products p USING(product_id)
INNER JOIN tbl_customers c USING(customer_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)

+---------------+--------------+-----------------------------------------------+--------------+
| customer_name |     name     |                  description                  | total_amount |
+---------------+--------------+-----------------------------------------------+--------------+
| Alice Johnson |    Laptop    |   High-performance laptop for professionals   |    120.50    |
|   Bob Smith   |  Headphones  |         Over-ear wireless headphones          |    75.25     |
| Charlie Brown |   Backpack   | Stylish and durable backpack for everyday use |    210.75    |
| Alice Johnson | Coffee Maker | Automatic drip coffee maker with programmable |    50.00     |
+---------------+--------------+-----------------------------------------------+--------------+
4 rows returned in time: (0.001 sec)



+---------------+--------------+-----------------------------------------------+--------------+
| customer_name |     name     |                  description                  | total_amount |


In [23]:
ut.select_all_query(table_name="tbl_customers", cursor_object=mysql_cursor)

join_query = """
SELECT customer_name, total_amount from tbl_customers
LEFT JOIN tbl_orders USING(customer_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)

+-------------+---------------+---------------------+--------------+--------------+-------------+
| customer_id | customer_name |        email        | phone_number |   address    |    city     |
+-------------+---------------+---------------------+--------------+--------------+-------------+
|      1      | Alice Johnson |  alice@example.com  | 123-456-7890 | 123 Main St  |  Cityville  |
|      2      |   Bob Smith   |   bob@example.com   | 987-654-3210 | 456 Oak Ave  | Townsville  |
|      3      | Charlie Brown | charlie@example.com | 555-123-4567 | 789 Pine ln  |  New York   |
|      4      |  David White  |  david@example.com  | 246-357-9753 | 567 Maple Dr |     NYC     |
|      5      |   Eva Black   |   eva@example.com   | 789-864-5791 | 890 Cedar Rd |  Big Apple  |
|      6      |  Frank Green  |  frank@example.com  | 579-468-375  |  123 Elm St  | Gotham City |
+-------------+---------------+---------------------+--------------+--------------+-------------+
6 rows returned in t

In [24]:
join_query = """
SELECT name, total_quantity from tbl_orders
RIGHT JOIN tbl_products USING(product_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)

+-----------------+----------------+
|      name       | total_quantity |
+-----------------+----------------+
|     Laptop      |       5        |
|   Headphones    |       3        |
|    Backpack     |       7        |
|  Coffee Maker   |       2        |
| Fitness Tracker |      NULL      |
+-----------------+----------------+
5 rows returned in time: (0.0 sec)



+-----------------+----------------+
|      name       | total_quantity |
+-----------------+----------------+
|     Laptop      |       5        |
|   Headphones    |       3        |
|    Backpack     |       7        |
|  Coffee Maker   |       2        |
| Fitness Tracker |      NULL      |
+-----------------+----------------+
5 rows returned in time: (0.001 sec)



+-----------------+----------------+
|      name       | total_quantity |
+-----------------+----------------+
|     Laptop      |       5        |
|   Headphones    |       3        |
|    Backpack     |       7        |
|  Coffee Maker   |       2       

In [25]:
ut.select_all_query(table_name="tbl_orders", cursor_object=mysql_cursor)

+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
| order_id | customer_id | product_id | total_quantity | total_amount | order_rating | length | width |   order_timestamp   | delivery_timestamp  |
+----------+-------------+------------+----------------+--------------+--------------+--------+-------+---------------------+---------------------+
|    1     |      1      |     1      |       5        |    120.50    |     4.5      |  2.30  | 1.80  | 2023-01-15 10:30:00 | 2023-01-16 15:45:00 |
|    2     |      2      |     2      |       3        |    75.25     |     3.8      |  1.50  | 1.20  | 2023-02-03 14:20:00 | 2023-02-05 11:10:00 |
|    3     |      3      |     3      |       7        |    210.75    |     4.2      |  2.80  | 2.00  | 2023-03-12 08:45:00 | 2023-03-14 09:30:00 |
|    4     |      1      |     4      |       2        |    50.00     |     4.0      |  1.80  | 1.50  | 2023-04-

In [26]:
# Full Outer Join combines the result of a left join and a right join

join_query = """
SELECT customer_name, total_quantity from tbl_customers 
FULL OUTER JOIN tbl_orders USING(customer_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)
print()

# Full Outer Join is not supported in mysql but can be achieved with the following syntax
join_query = """
SELECT customer_name, total_quantity from tbl_customers
LEFT JOIN tbl_orders USING(customer_id)
UNION
SELECT customer_name, total_quantity from tbl_customers 
RIGHT JOIN tbl_orders USING(customer_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)

+---------------+----------------+
| customer_name | total_quantity |
+---------------+----------------+
| Alice Johnson |       5        |
|   Bob Smith   |       3        |
| Charlie Brown |       7        |
| Alice Johnson |       2        |
|   Eva Black   |      NULL      |
|  Frank Green  |      NULL      |
|  David White  |      NULL      |
+---------------+----------------+
7 rows returned in time: (0.0 sec)



+---------------+----------------+
| customer_name | total_quantity |
+---------------+----------------+
| Alice Johnson |       5        |
| Alice Johnson |       2        |
|   Bob Smith   |       3        |
| Charlie Brown |       7        |
|  David White  |      NULL      |
|   Eva Black   |      NULL      |
|  Frank Green  |      NULL      |
+---------------+----------------+
7 rows returned in time: (0.0 sec)



+---------------+----------------+
| customer_name | total_quantity |
+---------------+----------------+
| Alice Johnson |       2        |
| Alice Johnso

In [27]:
# Full Outer Join combines the result of a left join and a right join

join_query = """
SELECT customer_name, name, total_quantity from tbl_customers 
FULL OUTER JOIN tbl_orders USING(customer_id)
FULL OUTER JOIN tbl_products USING(product_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=join_query, cursor_object=sqlite_cursor)
print()
# Full Outer Join is not supported in mysql but can be achieved with the following syntax
join_query = """
SELECT customer_name, name, total_quantity from tbl_customers
LEFT JOIN tbl_orders USING(customer_id)
LEFT JOIN tbl_products USING(product_id)
UNION
SELECT customer_name, name, total_quantity from tbl_customers 
RIGHT JOIN tbl_orders USING(customer_id)
RIGHT JOIN tbl_products USING(product_id);
"""
ut.execute_display_query_results(query=join_query, cursor_object=mysql_cursor)

+---------------+-----------------+----------------+
| customer_name |      name       | total_quantity |
+---------------+-----------------+----------------+
| Alice Johnson |     Laptop      |       5        |
|   Bob Smith   |   Headphones    |       3        |
| Charlie Brown |    Backpack     |       7        |
| Alice Johnson |  Coffee Maker   |       2        |
|   Eva Black   |      NULL       |      NULL      |
|  Frank Green  |      NULL       |      NULL      |
|  David White  |      NULL       |      NULL      |
|     NULL      | Fitness Tracker |      NULL      |
+---------------+-----------------+----------------+
8 rows returned in time: (0.0 sec)



+---------------+-----------------+----------------+
| customer_name |      name       | total_quantity |
+---------------+-----------------+----------------+
| Alice Johnson |     Laptop      |       5        |
| Alice Johnson |  Coffee Maker   |       2        |
|   Bob Smith   |   Headphones    |       3        |
| Charli

## Summary Statistics

In [28]:
ut.summary_statistics(table_name="tbl_orders", cursor_object=mysql_cursor)
ut.summary_statistics(table_name="tbl_products", cursor_object=mysql_cursor)

==================== SUMMARY STATISTICS FOR tbl_orders TABLE ====================
+----------+---------------------+---------------------+---------------------+---------------------+
| COUNT(*) | MAX(total_quantity) | MIN(total_quantity) | AVG(total_quantity) | SUM(total_quantity) |
+----------+---------------------+---------------------+---------------------+---------------------+
|    4     |          7          |          2          |       4.2500        |         17          |
+----------+---------------------+---------------------+---------------------+---------------------+
1 row returned in time: (0.0 sec)


+----------+-------------------+-------------------+-------------------+-------------------+
| COUNT(*) | MAX(total_amount) | MIN(total_amount) | AVG(total_amount) | SUM(total_amount) |
+----------+-------------------+-------------------+-------------------+-------------------+
|    4     |      210.75       |       50.00       |    114.125000     |      456.50       |
+----

In [29]:
ut.summary_statistics(table_name="tbl_orders", cursor_object=postgres_cursor)
ut.summary_statistics(table_name="tbl_products", cursor_object=postgres_cursor)

==================== SUMMARY STATISTICS FOR tbl_orders TABLE ====================
+-------+--------------------+--------------------+--------------------+--------------------+
| count | max_total_quantity | min_total_quantity | avg_total_quantity | sum_total_quantity |
+-------+--------------------+--------------------+--------------------+--------------------+
|   4   |         7          |         2          |       4.2500       |         17         |
+-------+--------------------+--------------------+--------------------+--------------------+
1 row returned in time: (0.001 sec)


+-------+------------------+------------------+------------------+------------------+
| count | max_total_amount | min_total_amount | avg_total_amount | sum_total_amount |
+-------+------------------+------------------+------------------+------------------+
|   4   |      210.75      |      50.00       |     114.1250     |      456.50      |
+-------+------------------+------------------+------------------+

In [30]:
ut.summary_statistics(table_name="tbl_orders", cursor_object=sqlite_cursor)
ut.summary_statistics(table_name="tbl_products", cursor_object=sqlite_cursor)

==================== SUMMARY STATISTICS FOR tbl_orders TABLE ====================
+----------+---------------------+---------------------+---------------------+---------------------+
| COUNT(*) | MAX(total_quantity) | MIN(total_quantity) | AVG(total_quantity) | SUM(total_quantity) |
+----------+---------------------+---------------------+---------------------+---------------------+
|    4     |          7          |          2          |        4.25         |         17          |
+----------+---------------------+---------------------+---------------------+---------------------+
1 row returned in time: (0.001 sec)


+----------+-------------------+-------------------+-------------------+-------------------+
| COUNT(*) | MAX(total_amount) | MIN(total_amount) | AVG(total_amount) | SUM(total_amount) |
+----------+-------------------+-------------------+-------------------+-------------------+
|    4     |      210.75       |        50         |      114.125      |       456.5       |
+--

In [31]:
ut.show_columns(table_name='tbl_orders', cursor_object=sqlite_cursor)

+-----+--------------------+----------------+---------+------------+----+
| cid |        name        |      type      | notnull | dflt_value | pk |
+-----+--------------------+----------------+---------+------------+----+
|  0  |      order_id      |    INTEGER     |    0    |    NULL    | 1  |
|  1  |    customer_id     |      INT       |    0    |    NULL    | 0  |
|  2  |     product_id     |      INT       |    0    |    NULL    | 0  |
|  3  |   total_quantity   |      INT       |    0    |    NULL    | 0  |
|  4  |    total_amount    | DECIMAL(10, 2) |    0    |    NULL    | 0  |
|  5  |    order_rating    |  DECIMAL(3,1)  |    0    |    NULL    | 0  |
|  6  |       length       | DECIMAL(5, 2)  |    0    |    NULL    | 0  |
|  7  |       width        | DECIMAL(5, 2)  |    0    |    NULL    | 0  |
|  8  |  order_timestamp   |   TIMESTAMP    |    0    |    NULL    | 0  |
|  9  | delivery_timestamp |   TIMESTAMP    |    0    |    NULL    | 0  |
+-----+--------------------+----------

In [32]:
query_string = """
SELECT DISTINCT ccu.column_name AS foreign_column_name 
FROM information_schema.columns as c 
LEFT JOIN information_schema.table_constraints as tc
ON tc.table_name = c.table_name
JOIN information_schema.constraint_column_usage AS ccu
ON ccu.constraint_name = tc.constraint_name
WHERE tc.table_name='tbl_orders' AND tc.constraint_type = 'FOREIGN KEY';
"""
ut.execute_display_query_results(query=query_string, cursor_object=postgres_cursor)
print()
query_string = """
SELECT DISTINCT ccu.column_name AS foreign_column_name 
FROM information_schema.columns as c 
LEFT JOIN information_schema.table_constraints as tc
ON tc.table_name = c.table_name
JOIN information_schema.constraint_column_usage AS ccu
ON ccu.constraint_name = tc.constraint_name
WHERE tc.table_name='tbl_products' AND tc.constraint_type = 'FOREIGN KEY';
"""
ut.execute_display_query_results(query=query_string, cursor_object=postgres_cursor)

+---------------------+
| foreign_column_name |
+---------------------+
|     customer_id     |
|     product_id      |
+---------------------+
2 rows returned in time: (0.009 sec)



+---------------------+
| foreign_column_name |
+---------------------+
|     category_id     |
+---------------------+
1 row returned in time: (0.006 sec)




## Case Statements 

```sql
SELECT
column_name, 
CASE 
WHEN condition1 THEN result1
WHEN condition2 THEN result2
ELSE default_result
END AS alias_name
FROM 
table_name
```

In [33]:
case_query = """
SELECT name, description, price,
CASE 
WHEN price <= 100 THEN 'Cheap'
WHEN price > 100 AND price < 500 THEN 'Affordable'
ELSE 'Expensive'
END AS ProductType
FROM tbl_products
"""
ut.execute_display_query_results(query=case_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=case_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=case_query, cursor_object=sqlite_cursor)

+-----------------+---------------------------------------------------------+--------+-------------+
|      name       |                       description                       | price  | producttype |
+-----------------+---------------------------------------------------------+--------+-------------+
|     Laptop      |        High-performance laptop for professionals        |  NULL  |  Expensive  |
|   Headphones    |              Over-ear wireless headphones               | 129.99 | Affordable  |
|    Backpack     |      Stylish and durable backpack for everyday use      | 49.99  |    Cheap    |
| Fitness Tracker | Water-resistant fitness tracker with heart rate monitor | 79.99  |    Cheap    |
|  Coffee Maker   |      Automatic drip coffee maker with programmable      | 550.00 |  Expensive  |
+-----------------+---------------------------------------------------------+--------+-------------+
5 rows returned in time: (0.0 sec)



+-----------------+----------------------------------

## Group By

In [34]:
group_query = """
SELECT city, count(*) AS Count
FROM tbl_customers
GROUP BY city;
"""
ut.execute_display_query_results(query=group_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=sqlite_cursor)

+-------------+-------+
|    city     | count |
+-------------+-------+
|  New York   |   1   |
| Townsville  |   1   |
|  Big Apple  |   1   |
| Gotham City |   1   |
|  Cityville  |   1   |
|     NYC     |   1   |
+-------------+-------+
6 rows returned in time: (0.0 sec)



+-------------+-------+
|    city     | Count |
+-------------+-------+
|  Cityville  |   1   |
| Townsville  |   1   |
|  New York   |   1   |
|     NYC     |   1   |
|  Big Apple  |   1   |
| Gotham City |   1   |
+-------------+-------+
6 rows returned in time: (0.001 sec)



+-------------+-------+
|    city     | Count |
+-------------+-------+
|  Big Apple  |   1   |
|  Cityville  |   1   |
| Gotham City |   1   |
|     NYC     |   1   |
|  New York   |   1   |
| Townsville  |   1   |
+-------------+-------+
6 rows returned in time: (0.0 sec)




In [35]:
group_query = """
SELECT c.name, c.category_id, count(p.product_id) AS Total
FROM tbl_categories AS c 
JOIN tbl_products AS p
ON c.category_id = p.category_id
GROUP BY c.category_id
HAVING COUNT(p.product_id) > 1;
"""
ut.execute_display_query_results(query=group_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=sqlite_cursor)

+-------------+-------------+-------+
|    name     | category_id | total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.001 sec)



+-------------+-------------+-------+
|    name     | category_id | Total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.001 sec)



+-------------+-------------+-------+
|    name     | category_id | Total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.0 sec)




In [36]:
group_query = """
SELECT c.name, c.category_id, count(p.product_id) AS Total
FROM tbl_categories AS c 
JOIN tbl_products AS p
ON c.category_id = p.category_id
GROUP BY c.category_id
HAVING COUNT(p.product_id) > 1;
"""
ut.execute_display_query_results(query=group_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=sqlite_cursor)

+-------------+-------------+-------+
|    name     | category_id | total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.001 sec)



+-------------+-------------+-------+
|    name     | category_id | Total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.001 sec)



+-------------+-------------+-------+
|    name     | category_id | Total |
+-------------+-------------+-------+
| Electronics |      1      |   3   |
+-------------+-------------+-------+
1 row returned in time: (0.0 sec)




In [37]:
ut.select_all_query(table_name="tbl_products", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=mysql_cursor)
print()
ut.select_all_query(table_name="tbl_products", cursor_object=sqlite_cursor)

+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
| product_id |      name       | price  |                       description                       |              tags               | category_id | supplier  |
+------------+-----------------+--------+---------------------------------------------------------+---------------------------------+-------------+-----------+
|     1      |     Laptop      |  NULL  |        High-performance laptop for professionals        |   electronics, portable, tech   |      1      | SupplierA |
|     2      |   Headphones    | 129.99 |              Over-ear wireless headphones               | electronics, audio, accessories |      1      | SupplierB |
|     3      |    Backpack     | 49.99  |      Stylish and durable backpack for everyday use      |  fashion, accessories, travel   |      2      |   NULL    |
|     5      | Fitness Tracker | 79.99  

In [38]:
ut.select_all_query(table_name="tbl_categories", cursor_object=postgres_cursor)
print()
ut.select_all_query(table_name="tbl_categories", cursor_object=mysql_cursor)
print()
ut.select_all_query(table_name="tbl_categories", cursor_object=sqlite_cursor)

+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitchen |
+-------------+------------------+
3 rows returned in time: (0.0 sec)



+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitchen |
+-------------+------------------+
3 rows returned in time: (0.0 sec)



+-------------+------------------+
| category_id |       name       |
+-------------+------------------+
|      1      |   Electronics    |
|      2      |     Clothing     |
|      3      | Home and Kitchen |
+-------------+------------------+
3 rows returned in time: (0.0 sec)




In [39]:
group_query = """
SELECT p.product_id, p.name, c.name
FROM tbl_products AS p 
JOIN tbl_categories AS c
ON c.category_id = p.category_id
WHERE c.name = 'Electronics';
"""
ut.execute_display_query_results(query=group_query, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=group_query, cursor_object=sqlite_cursor)

+------------+-----------------+-------------+
| product_id |      name       |    name     |
+------------+-----------------+-------------+
|     1      |     Laptop      | Electronics |
|     2      |   Headphones    | Electronics |
|     5      | Fitness Tracker | Electronics |
+------------+-----------------+-------------+
3 rows returned in time: (0.0 sec)



+------------+-----------------+-------------+
| product_id |      name       |    name     |
+------------+-----------------+-------------+
|     1      |     Laptop      | Electronics |
|     2      |   Headphones    | Electronics |
|     5      | Fitness Tracker | Electronics |
+------------+-----------------+-------------+
3 rows returned in time: (0.0 sec)



+------------+-----------------+-------------+
| product_id |      name       |    name     |
+------------+-----------------+-------------+
|     1      |     Laptop      | Electronics |
|     2      |   Headphones    | Electronics |
|     5      | Fitness Tracker 

# SQL Functions

## Mathematical Functions

In [40]:
query_str = "SELECT ABS(-2.6)"
ut.execute_display_query_results(query=query_str, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=sqlite_cursor)

+-----+
| abs |
+-----+
| 2.6 |
+-----+
1 row returned in time: (0.0 sec)



+-----------+
| ABS(-2.6) |
+-----------+
|    2.6    |
+-----------+
1 row returned in time: (0.0 sec)



+-----------+
| ABS(-2.6) |
+-----------+
|    2.6    |
+-----------+
1 row returned in time: (0.0 sec)




In [41]:
query_str = "SELECT CEIL(2.3)"
ut.execute_display_query_results(query=query_str, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=mysql_cursor)
print()
# CEIL is not compiled by default - and is only active if the amalgamation is compiled using the -DSQLITE_ENABLE_MATH_FUNCTIONS
# ut.execute_display_query_results(query=query_str, cursor_object=sqlite_cursor)

+------+
| ceil |
+------+
|  3   |
+------+
1 row returned in time: (0.0 sec)



+-----------+
| CEIL(2.3) |
+-----------+
|     3     |
+-----------+
1 row returned in time: (0.0 sec)





In [42]:
query_str = "SELECT FLOOR(2.3)"
ut.execute_display_query_results(query=query_str, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=mysql_cursor)
print()
# FLOOR is not compiled by default - and is only active if the amalgamation is compiled using the -DSQLITE_ENABLE_MATH_FUNCTIONS
# ut.execute_display_query_results(query=query_str, cursor_object=sqlite_cursor)

+-------+
| floor |
+-------+
|   2   |
+-------+
1 row returned in time: (0.0 sec)



+------------+
| FLOOR(2.3) |
+------------+
|     2      |
+------------+
1 row returned in time: (0.0 sec)





In [43]:
query_str = "SELECT ROUND(2.3456, 2) AS Rounded"
ut.execute_display_query_results(query=query_str, cursor_object=postgres_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=mysql_cursor)
print()
ut.execute_display_query_results(query=query_str, cursor_object=sqlite_cursor)

+---------+
| rounded |
+---------+
|  2.35   |
+---------+
1 row returned in time: (0.0 sec)



+---------+
| Rounded |
+---------+
|  2.35   |
+---------+
1 row returned in time: (0.0 sec)



+---------+
| Rounded |
+---------+
|  2.35   |
+---------+
1 row returned in time: (0.0 sec)




In [44]:
ut.display_all_tables_in_database(cursor_object=mysql_cursor)
ut.display_all_tables_in_database(cursor_object=postgres_cursor)
ut.display_all_tables_in_database(cursor_object=sqlite_cursor)

+----------------+----------------+---------------+
|   TABLE_NAME   | DATABASE NAME  | TABLE_CATALOG |
+----------------+----------------+---------------+
| tbl_categories | db_onlinestore |      def      |
| tbl_customers  | db_onlinestore |      def      |
|   tbl_orders   | db_onlinestore |      def      |
|  tbl_products  | db_onlinestore |      def      |
+----------------+----------------+---------------+
4 rows returned in time: (0.001 sec)


+----------------+--------------+
|   table_name   | table_schema |
+----------------+--------------+
| tbl_categories |    public    |
|  tbl_products  |    public    |
| tbl_customers  |    public    |
|   tbl_orders   |    public    |
+----------------+--------------+
4 rows returned in time: (0.001 sec)


+----------------+
|      name      |
+----------------+
| tbl_categories |
|  tbl_products  |
| tbl_customers  |
|   tbl_orders   |
+----------------+
4 rows returned in time: (0.0 sec)




In [45]:
#mysql_cursor.close()
#mysql_connection.close()
#postgres_cursor.close()
#postgres_connection.close()
#sqlite_cursor.close()
#sqlite_cursor.close()